In [1]:
from sklearn import svm, metrics, tree
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.linear_model import LogisticRegression, SGDClassifier
import autokeras as ak
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense,Activation,Dropout, MaxPooling1D, Conv1D, Flatten, BatchNormalization
# from tensorflow.keras.layers.normalization import BatchNormalization 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
# from xgboost import XGBClassifier
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import snappy
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
mask_file = r'\water_mask'
mask_path = r'D:\workspace\jupyterNotebooks\Remote Sensing\Water-Segmentation\data'
product_path = r'D:\Engi_Books\CDSAML\reflectance\data\subset_0_of_S1S2_collocated.dim'

In [3]:
water_veg_mask = np.load(r'.\data\water_veg_mask.npy')

In [4]:
unique, counts = np.unique(water_veg_mask, return_counts=True)
print(unique, counts)
water_veg_mask = water_veg_mask.flatten()

[0 1 2] [503834  14724 135923]


In [5]:
product = snappy.ProductIO.readProduct(product_path)
width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()

In [6]:
# B2 = product.getBand('B2')
# B3 = product.getBand('B3')
# B4 = product.getBand('B4')
# B5 = product.getBand('B5')
# B6 = product.getBand('B6')
# B7 = product.getBand('B7')
# B8 = product.getBand('B8')
# B8A = product.getBand('B8A')
# B11 = product.getBand('B11')
B12 = product.getBand('B12_S')

In [7]:
# B2_pixels = np.zeros(width * height, np.float32)
# B2.readPixels(0,0,width,height,B2_pixels)
# # B2_pixels.flatten()

# B3_pixels = np.zeros(width * height, np.float32)
# B3.readPixels(0,0,width,height,B3_pixels)
# # B3_pixels.flatten()

# B4_pixels = np.zeros(width * height, np.float32)
# B4.readPixels(0,0,width,height,B4_pixels)
# # B4_pixels.flatten()

# B5_pixels = np.zeros(width * height, np.float32)
# B5.readPixels(0,0,width,height,B5_pixels)

# B6_pixels = np.zeros(width * height, np.float32)
# B6.readPixels(0,0,width,height,B6_pixels)

# B7_pixels = np.zeros(width * height, np.float32)
# B7.readPixels(0,0,width,height,B7_pixels)

# B8_pixels = np.zeros(width * height, np.float32)
# B8.readPixels(0,0,width,height,B8_pixels)
# # B8_pixels.flatten()

# B8A_pixels = np.zeros(width * height, np.float32)
# B8A.readPixels(0,0,width,height,B8A_pixels)

# B11_pixels = np.zeros(width * height, np.float32)
# B11.readPixels(0,0,width,height,B11_pixels)

B12_pixels = np.zeros(width * height, np.float32)
B12.readPixels(0,0,width,height,B12_pixels)

array([0.1214, 0.1363, 0.1363, ..., 0.2204, 0.2131, 0.2131], dtype=float32)

In [8]:
# print(water_veg_mask.shape)
# print(B2_pixels.shape)
# print(B3_pixels.shape)
# print(B8_pixels.shape)
# print(B8_pixels.shape)

In [9]:
df = pd.DataFrame({
#     'B2':B2_pixels, 
#                    'B3': B3_pixels,
#                    'B4': B4_pixels,
#                    'B5': B5_pixels,
#                    'B6': B6_pixels,
#                    'B7': B7_pixels,
#                    'B8': B8_pixels,
#                    'B8A': B8A_pixels,
#                    'B11': B11_pixels,
                   'B12': B12_pixels,
                    'Y':water_veg_mask})
#df.drop(df[df.X == 0].index,inplace=True)

In [10]:
df.head()

,B12,Y
0,0.1214,2
1,0.1363,2
2,0.1363,2
3,0.1508,0
4,0.1508,0


In [11]:
x_cols = ['B12']

In [12]:
sample_0 = df.query('Y == 0').sample(counts[1])
sample_1 = df.query('Y == 1').sample(counts[1])
sample_2 = df.query('Y == 2').sample(counts[1])

In [13]:
sampled_df = pd.concat([sample_0, sample_1, sample_2],keys=['B12', 'Y'])
sampled_df = sampled_df.sample(frac=1).reset_index(drop=True)
sampled_df.describe()

# sampled_df = pd.DataFrame()
# sampled_df = sampled_df.append(sample_0, ignore_index = True).append(sample_1, ignore_index = True)
# sampled_df = sampled_df.sample(frac=1).reset_index(drop=True)
# sampled_df.describe()

,B12,Y
count,29448.000000,29448.000000
mean,0.106167,0.500000
std,0.105833,0.500008
min,0.002200,0.000000
25%,0.007000,0.000000
50%,0.078500,0.500000
75%,0.194000,1.000000
max,0.978100,1.000000


In [20]:
X_train,X_test,y_train,y_test = train_test_split(sampled_df[x_cols],sampled_df['Y'],test_size=0.2,random_state=1234, stratify = sampled_df['Y'])

In [21]:
# X_test,X_val, y_test, y_val = train_test_split(X_test_val,y_test_val,test_size=0.5,random_state=4321, stratify = y_test_val)

In [22]:
def show_confusion_matrix(y_test, y_pred, score):    
    cm = metrics.confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(9,9))
    sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');
    all_sample_title = 'Accuracy Score: {0}'.format(score)
    plt.title(all_sample_title, size = 15);
    return cm

In [23]:
y_train= to_categorical(y_train,num_classes=3)
y_test= to_categorical(y_test,num_classes=3)
# y_val = to_categorical(y_val,num_classes=3)
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)
print("Shape of y_val",y_val.shape)

Shape of y_train (23558, 3)
Shape of y_test (5890, 3)


NameError: name 'y_val' is not defined

In [24]:
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=200)

In [25]:
clf.fit(
    X_train,
    y_train,
    # Use your own validation set.
    validation_data=(X_test, y_test),
    epochs=100,
)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |?                 
structured_data...|2                 |?                 
structured_data...|False             |?                 
structured_data...|0                 |?                 
structured_data...|32                |?                 
structured_data...|32                |?                 
classification_...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 

Invalid model 0/5
Invalid model 1/5


Traceback (most recent call last):
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\keras_tuner\engine\hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\keras_tuner\engine\hypermodel.py", line 84, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\autokeras\graph.py", line 250, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\autokeras\engine\block.py", line 38, in _build_wrapper
    return super()._build_wrapper(hp, *args, **kwargs)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\keras_tuner\engine\hypermodel.py", line 84, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\autokeras\blocks\wrapper.py", line 251, in buil

Invalid model 2/5
Invalid model 3/5
Invalid model 4/5
Invalid model 5/5


  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\keras_tuner\engine\hyperparameters.py", line 928, in Boolean
    return self._retrieve(hp)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\keras_tuner\engine\hyperparameters.py", line 707, in _retrieve
    return self.values[hp.name]
KeyError: 'structured_data_block_2/normalize'
Traceback (most recent call last):
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\keras_tuner\engine\hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\keras_tuner\engine\hypermodel.py", line 84, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\autokeras\graph.py", line 250, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "C:\Users\shubh\AppData\Roaming\Python\Python36\site-packages\autokeras\engine\block.py", lin

RuntimeError: Too many failed attempts to build model.

In [ ]:
clf_y_pred = clf.predict(X_val)
target_names = ['class 0', 'class 1', 'class 2']

In [ ]:
print(classification_report(y_val, clf_y_pred, target_names=target_names))

In [ ]:
ak_eval = clf.evaluate(X_val,y_val)

In [ ]:
print(ak_eval)

In [ ]:
y_label=np.argmax(y_val,axis=1)
length=len(clf_y_pred)
predict_label_ak=np.argmax(clf_y_pred,axis=1)

accuracy_ak=np.sum(y_label==predict_label_ak)/length * 100 

cm = show_confusion_matrix(y_test = y_label, y_pred = predict_label_ak, score = accuracy_ak)

In [ ]:
model = clf.export_model()
model.summary()
# print(x_train.dtype)
# # numpy array in object (mixed type) is not supported.
# # convert it to unicode.
# model.predict(x_train.astype(np.unicode))